# Pipelines: Metaflow model training

##  Install dependencies

In [ ]:
# docker run -p 8888:8888 quay.io/jupyter/scipy-notebook

In [1]:
%pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install metaflow==2.13.9 pandas==2.2.3 numpy==1.26.4 scikit-learn==1.6.1 optuna==4.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 16.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 16.4 MB/s eta 0:00:0000:010:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


## Set username

In [4]:
# Set username for workflows
import os
os.environ["USERNAME"] = "pablo"

In [12]:
%%writefile metaflow_trainingflow.py
from metaflow import FlowSpec, Parameter, step

class TrainingFlow(FlowSpec):
    # Define the parameters for the flow
    max_depth = Parameter('max_depth', default=2, help='Max depth of the random forest classifier')
    n_estimators = Parameter('n_estimators', default=100, help='Number of estimators for the random forest classifier')
    random_state = Parameter('random_state', default=0, help='Random state for the random forest classifier')
    
    @step
    def start(self):
        # Start the flow
        self.next(self.ingest_data)
        
    @step
    def ingest_data(self):
        from sklearn.datasets import load_iris
    
        # Load the iris dataset
        iris = load_iris()
        
        #pylint: disable=no-member
        self.X = iris.data
        self.y = iris.target
        #pylint: enable=no-member
        
        self.next(self.split_data)

    @step
    def split_data(self):
        # Split the data into train and test
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=self.random_state)
        
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        
        self.next(self.train)

    @step
    def train(self):
        # Train the model
        from sklearn.ensemble import RandomForestClassifier
        
        # Create the random forest classifier
        self.rf = RandomForestClassifier(max_depth=self.max_depth, n_estimators=self.n_estimators, random_state=self.random_state)
        # Train the model
        self.rf.fit(self.X_train, self.y_train)
        
        self.next(self.show_metrics)

    @step
    def show_metrics(self):
        # Print some metrics
        from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
        
        y_pred = self.rf.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, y_pred)
        report = classification_report(self.y_test, y_pred)
        confusion = confusion_matrix(self.y_test, y_pred)
        
        print(f'Accuracy: {accuracy}')
        print(f'Report:\n{report}')
        print(f'Confusion matrix:\n{confusion}')

        self.next(self.register_model)
        
    @step
    def register_model(self):
        # Save the model in a pickle file in local storage
        import pickle
        with open('model.pkl', 'wb') as f:
            pickle.dump(self.rf, f)
        
        self.next(self.end)
        
    @step
    def end(self):
        pass
    
if __name__ == '__main__':
    TrainingFlow()

Overwriting metaflow_trainingflow.py


In [14]:
!python metaflow_trainingflow.py run --max_depth 2 --n_estimators 100 --random_state 0

Metaflow 2.13.9 executing TrainingFlow for user:pablo
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
2025-03-20 18:59:11.409 Workflow starting (run-id 1742497151406177):
2025-03-20 18:59:11.416 [1742497151406177/start/1 (pid 3712)] Task is starting.
2025-03-20 18:59:11.622 [1742497151406177/start/1 (pid 3712)] Task finished successfully.
2025-03-20 18:59:11.625 [1742497151406177/ingest_data/2 (pid 3714)] Task is starting.
2025-03-20 18:59:12.693 [1742497151406177/ingest_data/2 (pid 3714)] Task finished successfully.
2025-03-20 18:59:12.698 [1742497151406177/split_data/3 (pid 3771)] Task is starting.
2025-03-20 18:59:13.698 [1742497151406177/split_data/3 (pid 3771)] Task finished successfully.
2025-03-20 18:59:13.717 [1742497151406177/train/4 (pid 3836)] Task is starting.
2025-03-20 18:59:14.887 [1742497151406177/train/4 (pid 3836)] Task finished successfully.
2025-03-20 18:59:14.891 [1742497151406177/show_metrics/